
# Exercise 13: Structured light

Solutions for the week 13 exercises. We follow the lecture terminology on structured light, phase shifting, and stereo triangulation.



## Exercise 13.1
If the cameras were not calibrated we would first estimate their intrinsic parameters using a planar calibration target.
Multiple views of a known pattern, e.g. a checkerboard, allow us to recover the intrinsic camera matrix $K$ and distortion coefficients by minimizing the reprojection error.
The relative rotation $R$ and translation $t$ (extrinsics) between the cameras can then be obtained with stereo calibration once the intrinsics are known.
This follows the standard pinhole model used throughout the lectures.



## Exercise 13.2
The images are first undistorted and rectified so that epipolar lines align horizontally.
The code below loads the calibration dictionary, prepares rectification maps, and stores the processed frames in two lists `ims0` and `ims1`.


In [0]:
import numpy as np
import cv2
from pathlib import Path
from matplotlib import pyplot as plt

data_dir = Path('exercises/ex13_data')
c = np.load(data_dir / 'calib.npy', allow_pickle=True).item()
im0 = cv2.imread(str(data_dir / 'sequence/frames0_0.png'))
size = (im0.shape[1], im0.shape[0])
stereo = cv2.stereoRectify(c['K0'], c['d0'], c['K1'], c['d1'], size, c['R'], c['t'], flags=0)
R0, R1, P0, P1 = stereo[:4]
maps0 = cv2.initUndistortRectifyMap(c['K0'], c['d0'], R0, P0, size, cv2.CV_32FC2)
maps1 = cv2.initUndistortRectifyMap(c['K1'], c['d1'], R1, P1, size, cv2.CV_32FC2)

ims0, ims1 = [], []
for idx in range(26):
    im0 = cv2.imread(str(data_dir / f'sequence/frames0_{idx}.png'), cv2.IMREAD_GRAYSCALE).astype(np.float32)
    im1 = cv2.imread(str(data_dir / f'sequence/frames1_{idx}.png'), cv2.IMREAD_GRAYSCALE).astype(np.float32)
    ims0.append(cv2.remap(im0, *maps0, cv2.INTER_LINEAR))
    ims1.append(cv2.remap(im1, *maps1, cv2.INTER_LINEAR))

plt.figure(figsize=(8,4))
plt.subplot(1,2,1); plt.imshow(ims0[0], cmap='gray'); plt.title('cam 0')
plt.subplot(1,2,2); plt.imshow(ims1[0], cmap='gray'); plt.title('cam 1')
plt.show()



## Exercise 13.3
We unwrap the phase for each camera using the heterodyne principle.
The function below follows the lecture derivation: Fourier analysis obtains the wrapped primary and secondary phases, the order of the primary phase is found from the phase cue, and the absolute phase is recovered.


In [0]:
def unwrap(ims, n_primary=16, n_secondary=8):
    prim = np.stack(ims[2:2+n_primary], axis=0)
    sec = np.stack(ims[18:18+n_secondary], axis=0)
    fft_p = np.fft.rfft(prim, axis=0)
    fft_s = np.fft.rfft(sec, axis=0)
    theta_p = np.angle(fft_p[1])
    theta_s = np.angle(fft_s[1])
    theta_c = theta_p - theta_s
    o_p = np.round((theta_c + np.pi) / (2*np.pi))
    theta = theta_p + 2*np.pi*o_p
    return theta

theta0 = unwrap(ims0)
theta1 = unwrap(ims1)
plt.imshow(theta0, cmap='hsv'); plt.title('theta0'); plt.colorbar(); plt.show()



## Exercise 13.4
A binary mask discards pixels with insufficient projector illumination.
We threshold the difference between the fully-on and fully-off images.


In [0]:
mask0 = (ims0[0] - ims0[1]) > 15
mask1 = (ims1[0] - ims1[1]) > 15
plt.figure(figsize=(8,4))
plt.subplot(1,2,1); plt.imshow(mask0, cmap='gray'); plt.title('mask0')
plt.subplot(1,2,2); plt.imshow(mask1, cmap='gray'); plt.title('mask1')
plt.show()



## Exercise 13.5
Rectified images allow correspondence search along rows. For each valid pixel in camera 0 we find the pixel in camera 1 with closest phase and record the disparity.


In [0]:
h, w = theta0.shape
q0s, q1s = [], []
disparity = np.zeros_like(theta0)
for i in range(h):
    for j0 in range(w):
        if not mask0[i, j0]:
            continue
        phases = theta1[i, mask1[i]]
        js = np.where(mask1[i])[0]
        j1 = js[np.argmin(np.abs(phases - theta0[i, j0]))]
        q0s.append([j0, i])
        q1s.append([j1, i])
        disparity[i, j0] = j0 - j1
plt.imshow(disparity, cmap='inferno'); plt.title('disparity'); plt.colorbar(); plt.show()



## Exercise 13.6
Using the projection matrices of the rectified stereo pair we triangulate the matched pixels to obtain a 3D point cloud in the camera 0 coordinate frame.


In [0]:
q0 = np.array(q0s, dtype=np.float32).T
q1 = np.array(q1s, dtype=np.float32).T
Q = cv2.triangulatePoints(P0, P1, q0, q1)
Q = cv2.convertPointsFromHomogeneous(Q.T)[:,0]
Q = Q[Q[:,2] > 0]  # keep points in front of cameras
# Visualization (requires Open3D)
# import open3d as o3d
# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(Q)
# o3d.visualization.draw_geometries([pcd])



## Exercise 13.7 (optional)
Color information can be added by sampling the rectified color images at the matched pixel locations and assigning these colors to the 3D points before visualization.



## Exercise 13.8 (optional)
Sub-pixel matching refines the disparity by linearly interpolating the phase responses along the epipolar line to locate the phase agreement with fractional pixel precision.
